# BBO-Rietveld テンプレート（YOUR_MATERIAL）

このノートブックは、ご自身の材料データに対してブラックボックス最適化（Optuna）でGSAS-IIのリートベルト精密化を自動化するためのテンプレートです。

- まず「設定」セルで STUDY_NAME や入出力ディレクトリ、初期プロジェクト（.gpx）などを調整してください。
- 次に、objective 関数内に探索空間と精密化の手順を実装してください（Y2O3/DSMO/LiCoO2 のノートを参考）。
- 実行は上から順にセルを実行します。

参考ドキュメント
- JupyterLab: https://jupyterlab.readthedocs.io/en/stable/
- GSASIIscriptable: https://gsas-ii.readthedocs.io/en/latest/GSASIIscriptable.html
- Optuna: https://optuna.readthedocs.io/en/stable/


In [ ]:
# 必要パッケージの読み込み
%matplotlib inline

import os
import sys
from multiprocessing import Process, Queue
import pandas as pd
import optuna
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# 注意: GSAS-II は環境にインストール済みのため、sys.path.append は不要です
# 使う箇所（クラスや関数内）で `import GSASIIscriptable as G2sc` を行います


In [ ]:
# 設定セル（まずここをあなたの材料に合わせて編集）
# STUDY_NAME: 解析対象の材料名（出力フォルダ名にも使用）
# RANDOM_SEED: Optuna の乱数シード（再現性確保）
# DATA_DIR: 入力データ (初期 gpx, PRM, CIF, CSV 等) を置いたフォルダ
# WORK_DIR: 最適化過程で生成される *.gpx や履歴DB等の出力先

STUDY_NAME = 'YOUR_MATERIAL'
RANDOM_SEED = 1024

DATA_DIR = 'data/' + STUDY_NAME
WORK_DIR = 'work/' + STUDY_NAME


In [ ]:
# make directories
! rm -f $WORK_DIR/$STUDY_NAME*
! mkdir -p $WORK_DIR

In [ ]:
class ProjectBBO:
    def __init__(self, trial_number):
        import GSASIIscriptable as G2sc
        import shutil
        
        # Create a project with a default project name
        ### Change here ###
        shutil.copyfile(DATA_DIR+'/'+'YOUR_PROJECT_FILE.gpx', 
                        WORK_DIR+'/'+'BBO_seed{0}_trial_{1}.gpx'.format(RANDOM_SEED, trial_number))
        
        self.gpx = G2sc.G2Project(gpxfile=os.path.join(WORK_DIR, 'BBO_seed{0}_trial_{1}.gpx'.format(RANDOM_SEED, trial_number)))

        # Add two histograms to the project
        self.hist1 = self.gpx.histograms()[0]
        self.phase0 = self.gpx.phases()[0]
        self.hist1.data['Instrument Parameters'][0]['I(L2)/I(L1)'] = [0.5, 0.5, 0]

        # Set to use iso
        for val in self.phase0.data['Atoms']:
            val[9] = 'I'

    def refine_and_calc_Rwp(self, param_dict):
        self.gpx.do_refinements([param_dict])
        for hist in self.gpx.histograms():
            _, Rwp = hist.name, hist.get_wR()
        return Rwp

In [ ]:
def objective(trial):
    """
    目的関数（Optuna）

    ここに探索空間の定義と GSAS-II を用いたリートベルト精密化の処理を書きます。
    以下はテンプレートです。Y2O3 / DSMO / LiCoO2 のノートを参考に実装してください。

    Parameters
    ----------
    trial : optuna.trial.Trial

    Returns
    -------
    Rwp : float
        小さいほど良い目的値（加重プロファイルR因子）
    """
    raise NotImplementedError(
        "objective(trial) をあなたのデータ処理に合わせて実装してください。"
    )


In [ ]:
# Optuna スタディの作成（履歴は WORK_DIR のSQLiteに保存）
study = optuna.create_study(
    study_name=f"{STUDY_NAME}_seed{RANDOM_SEED}",
    storage=f"sqlite:///{WORK_DIR}/history_sqlite.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(n_startup_trials=20, seed=RANDOM_SEED),
)


最適化を実行する前に objective 関数を実装してください。試行回数 (n_trials) はデータの難易度と計算時間に応じて調整します。例: 50, 100, 200 など。


In [ ]:
# 最適化実行（objective を実装した後で実行）
# study.optimize(objective, n_trials=100)


In [ ]:
# 結果の整形（Rwp最小の試行が上に来るようソート）
df = study.trials_dataframe()
df.columns = [''.join(col).replace('params', '').strip() for col in df.columns.values]
df.rename(columns={'value': 'Rwp', 'number': 'trial'}, inplace=True)
df.sort_values('Rwp')


In [ ]:
# Best configuration
study.best_params

In [ ]:
# Best Rwp
study.best_value

In [ ]:
# 成果物の保存（任意）
import json

# 全試行履歴をCSVとして保存
df.to_csv(f"{WORK_DIR}/trials.csv", index=False)

# 最良試行のパラメータをJSON保存
with open(f"{WORK_DIR}/best_params.json", "w") as f:
    json.dump(study.best_params, f, indent=2)

# 参考: 最良試行のGPXからCIFを書き出す（必要に応じて有効化）
# from GSASIIscriptable import G2Project
# gpx = G2Project(f"{WORK_DIR}/{STUDY_NAME}_seed{RANDOM_SEED}_trial_{study.best_trial.number}.gpx")
# phase = gpx.phases()[0]
# phase.export_CIF(f"{WORK_DIR}/{STUDY_NAME}_best.cif", quickmode=True)

In [ ]:
# Rwp plot
def rwp_plot():
    minvalues = [df.iloc[0]['Rwp']]
    for i in range(1, df.shape[0]):
        minvalues.append(min(minvalues[-1], df.iloc[i]['Rwp']))
    minvalues = pd.DataFrame(minvalues)
    
    minvalues.plot(legend=None)
#     plt.ylim([6, 16])
    plt.grid(color='#cccccc')
    plt.ylabel('$R_{wp}$')
    plt.xlabel('Number of trials')
    plt.show()
    
rwp_plot()

In [ ]:
# Rietveld plot
def rietveld_plot():
    import GSASIIscriptable as G2sc

    gpx = G2sc.G2Project(
        '%s/%s_seed%s_trial_%s.gpx' % (WORK_DIR, STUDY_NAME, RANDOM_SEED, study.best_trial.number))

    hist1 = gpx.histograms()[0]
    phase0 = gpx.phases()[0]

    hist = hist1
    i = 5
    two_theta = hist.getdata("X")[::i]
    Yobs = hist.getdata("Yobs")[::i]
    Ycalc = hist.getdata("Ycalc")[::i]
    bg = hist.getdata("Background")[::i]
    residual = hist.getdata("Residual")[::i]

    fig = plt.figure()
    gs = GridSpec(5, 1, figure=fig)
    ax1 = fig.add_subplot(gs[:4, :])
    ax2 = fig.add_subplot(gs[4, :])
    fig.subplots_adjust(hspace=0)
    ax1.grid(color='#cccccc')

    ax1.scatter(two_theta, Yobs, marker='P', lw=0.0001, c='Black', label='XRD (Obs)')
    ax1.plot(two_theta, Ycalc, label='XRD (Calc)')
    ax1.plot(two_theta, bg, color='red', label='Background (Calc)')
    ax1.set_ylabel('Intensity')
    ax1.legend()
    ax2.plot(two_theta, residual, color='blue')
    plt.setp(ax1.get_xticklabels(), visible=False);
    # ax2.set_ylim(-6600, 6600)
    plt.xlabel(r'$2\theta$ (deg.)')
    ax2.set_ylabel('Residual')
    # change 2theta range according to your data
    ax1.set_xlim(15, 150)
    ax2.set_xlim(15, 150)
    plt.show()
    
rietveld_plot()